Data Cleaning

In [1]:
import numpy as np
import pandas as pd


We start by reading data in from cal_fire_data CSV file (downloaded from CAL FIRE database):

In [2]:
cal_fire_data = pd.read_csv('../data/cal_fire_data.csv')
cal_fire_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130722 entries, 0 to 130721
Data columns (total 46 columns):
 #   Column                                                        Non-Null Count   Dtype  
---  ------                                                        --------------   -----  
 0   OBJECTID                                                      130722 non-null  int64  
 1   * Damage                                                      130722 non-null  object 
 2   * Street Number                                               126302 non-null  float64
 3   * Street Name                                                 125236 non-null  object 
 4   * Street Type (e.g. road, drive, lane, etc.)                  116260 non-null  object 
 5   Street Suffix (e.g. apt. 23, blding C)                        62017 non-null   object 
 6   * City                                                        98991 non-null   object 
 7   State                                                   

/var/folders/l1/qgkmnkpx453gnydcmvkyxhh80000gn/T/ipykernel_69411/3475726481.py:1: DtypeWarning: Columns (12,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  cal_fire_data = pd.read_csv('../data/cal_fire_data.csv')


We remove the columns that have unusable or repeated information:

In [3]:
cal_fire_data = cal_fire_data.drop(columns=['GLOBALID', 'OBJECTID', 'Zip Code', '* Street Name',
                                '* Street Number', 'Site Address (parcel)', 'x', 'y',
                                'Incident Number (e.g. CAAEU 123456)', 'Battalion',
                                'State', 'APN (parcel)', 'If Affected 1-9% - Where did fire start?',
                                'If Affected 1-9% - What started fire?', 'Structure Category',
                                '* Street Type (e.g. road, drive, lane, etc.)', 'Street Suffix (e.g. apt. 23, blding C)',
                                '* City', '* CAL FIRE Unit', 'County', 'Community',
                                'Structure Defense Actions Taken', '# Units in Structure (if multi unit)',
                                '# of Damaged Outbuildings < 120 SQFT', '# of Non Damaged Outbuildings < 120 SQFT',
                                'Distance - Propane Tank to Structure', 'Distance - Residence to Utility/Misc Structure &gt; 120 SQFT',
                                'Fire Name (Secondary)'])


We remove the 2 rows where the incident was not Fire, and then the column Hazard Type:

In [4]:
cal_fire_data.drop(cal_fire_data.index[cal_fire_data['Hazard Type'] != 'Fire'], inplace = True)
cal_fire_data = cal_fire_data.drop(columns=['Hazard Type'])


We clean Structure Type values:

In [5]:
cal_fire_data['* Structure Type'] = cal_fire_data['* Structure Type'].replace({'Single Famliy Residence Single Story': 'Single Family Residence Single Story'})


We clean various unknown values:

In [6]:
cal_fire_data['* Roof Construction'] = cal_fire_data['* Roof Construction'].replace({'Unknown': np.nan})
cal_fire_data['* Eaves'] = cal_fire_data['* Eaves'].replace({'Unknown': np.nan})
cal_fire_data['* Vent Screen'] = cal_fire_data['* Vent Screen'].replace({'Unknown': np.nan})
cal_fire_data['* Exterior Siding'] = cal_fire_data['* Exterior Siding'].replace({'Unknown': np.nan})
cal_fire_data['* Window Pane'] = cal_fire_data['* Window Pane'].replace({'Unknown': np.nan})
cal_fire_data['* Deck/Porch On Grade'] = cal_fire_data['* Deck/Porch On Grade'].replace({'Unknown': np.nan})
cal_fire_data['* Deck/Porch Elevated'] = cal_fire_data['* Deck/Porch Elevated'].replace({'Unknown': np.nan})
cal_fire_data['* Patio Cover/Carport Attached to Structure'] = cal_fire_data['* Patio Cover/Carport Attached to Structure'].replace({'Unknown': np.nan})
cal_fire_data['* Fence Attached to Structure'] = cal_fire_data['* Fence Attached to Structure'].replace({'Unknown': np.nan})


We create a new row with the building's age (transformed from year built):

In [7]:
cal_fire_data['Age'] = 2025 - cal_fire_data['Year Built (parcel)']
cal_fire_data = cal_fire_data.drop(columns=['Year Built (parcel)'])


The column Incident Start Date includes badly recorded hours, so we clean it up to only include a date.
The date now always appears in DD/MM/YYYY format.

In [8]:
# Cleans Incident Start Date to appear in DD/MM/YYYY format.
def cleandate(date):
    date = date[:date.find(' ')]
    month = date[:date.find('/')]
    year = date[-4:]
    day = date[date.find('/')+1:-5]
    if len(month) == 1:
        month = '0' + month
    if len(day) == 1:
        day = '0' + day
    return(month + '/' + day + '/' + year) 

cal_fire_data['Incident Start Date'] = cal_fire_data['Incident Start Date'].apply(cleandate)


We make a new column that has the date without the day:

In [9]:
# Creates a new column that has the date without the day.
def dayremove(date):
    month = date[:date.find('/')]
    year = date[-4:]
    return(month + '/' + year)

cal_fire_data['Incident Month/Year'] = cal_fire_data['Incident Start Date'].apply(dayremove)
cal_fire_data = cal_fire_data.drop(columns=['Incident Start Date'])


We removed observations where the incident occurred before 2018, since buildings without damage weren't reported before then and they could bias the data:

In [10]:
# Gets observations where the incident occurred before 2018
def getyear(date):
    return int(date[-2:])

cal_fire_data['Year'] = cal_fire_data['Incident Month/Year'].apply(getyear)
cal_fire_data = cal_fire_data[cal_fire_data['Year'] > 17]
cal_fire_data = cal_fire_data.drop(columns=['Year', 'Incident Month/Year'])


We remove the rows where Fire Damage was Inaccessible. We then create a new column, Destroyed, that 
converts Damage levels into either Not Destroyed (0) or Destroyed (1).

In [11]:
cal_fire_data.drop(cal_fire_data.index[cal_fire_data['* Damage'] == 'Inaccessible'], inplace = True)
cal_fire_data['Destroyed'] = cal_fire_data['* Damage'].replace({'No Damage': 0, 'Affected (1-9%)': 0, 'Minor (10-25%)': 0, 'Major (26-50%)': 0, 'Destroyed (>50%)': 1})
cal_fire_data = cal_fire_data.drop(columns=['* Damage'])


/var/folders/l1/qgkmnkpx453gnydcmvkyxhh80000gn/T/ipykernel_69411/3483222767.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cal_fire_data['Destroyed'] = cal_fire_data['* Damage'].replace({'No Damage': 0, 'Affected (1-9%)': 0, 'Minor (10-25%)': 0, 'Major (26-50%)': 0, 'Destroyed (>50%)': 1})


We convert Age values of 2024 or 2025 to appear as missing instead:

In [12]:
# Finds age values of 2024/2025
def missingyear(age):
    if age > 1000.0:
        return np.nan
    return age

cal_fire_data['Age'] = cal_fire_data['Age'].apply(missingyear)


We convert empty string values to appear as missing instead:

In [13]:
cal_fire_data = cal_fire_data.replace({' ':np.nan})


We drop all observations with missing values:

In [14]:
cal_fire_data = cal_fire_data.dropna()


We delete remaining observations with mislabeled data:

In [15]:
#Roof Construction Cleanup
cal_fire_data = cal_fire_data.drop(index = 95088)
cal_fire_data = cal_fire_data.drop(index = 101186)

#Eaves Cleanup
cal_fire_data = cal_fire_data.drop(index = 114021)
cal_fire_data = cal_fire_data.drop(index = 114283)

#Vent Screen Cleanup
cal_fire_data = cal_fire_data.drop(index = 94177)
cal_fire_data = cal_fire_data.drop(index = 97951)
cal_fire_data = cal_fire_data.drop(index = 105779)

#Window Pane Cleanup
cal_fire_data = cal_fire_data.drop(index = 96139)

cal_fire_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 42789 entries, 0 to 130717
Data columns (total 16 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   * Incident Name                              42789 non-null  object 
 1   * Structure Type                             42789 non-null  object 
 2   * Roof Construction                          42789 non-null  object 
 3   * Eaves                                      42789 non-null  object 
 4   * Vent Screen                                42789 non-null  object 
 5   * Exterior Siding                            42789 non-null  object 
 6   * Window Pane                                42789 non-null  object 
 7   * Deck/Porch On Grade                        42789 non-null  object 
 8   * Deck/Porch Elevated                        42789 non-null  object 
 9   * Patio Cover/Carport Attached to Structure  42789 non-null  object 
 10  * 

We save the cleaned data to the cal_fire_data_test CSV file for test data (data from the 3 most recent incidents) and to the cal_fire_data_train CSV file for training data (the rest of the data):

In [16]:
cal_fire_data_test = cal_fire_data[cal_fire_data['* Incident Name'].isin(['Franklin','Eaton','Palisades'])]
cal_fire_data_train = cal_fire_data[~cal_fire_data.index.isin(cal_fire_data_test.index)]

cal_fire_data_train.to_csv('../data/cal_fire_data_train.csv', index=False)
cal_fire_data_test.to_csv('../data/cal_fire_data_test.csv', index=False)
